# Course Recommender System

In [1]:
from collections import defaultdict
from itertools import combinations
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from coursemate.dataset import Dataset

In [2]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
%load_ext autoreload
%autoreload 2

## 0. Dataset preparation

In [16]:
dataset = Dataset('../data/Coursera_courses.csv', '../data/Coursera.csv', '../data/Coursera_reviews.csv')
dataset.set_interaction_counts(3, 50)
dataset.show_dataset_details()

Loading Coursera courses...
Loading Coursera reviews...
Segmenting out students with less than 3 or more than 50 reviews...
30719 students, 468 courses, 174219 reviews
Sparsity: 1.21%
Duplicates: 4.54%


In [17]:
dataset.set_train_test_split_by_user()

# For the content-based and CF recommender
train_Xmatrix, train_ymatrix, df_train_X, df_train_y = dataset.get_train_matrix_split(ratio=0.8)
test_Xmatrix, test_ymatrix, df_test_X, df_test_y = dataset.get_test_matrix_split(ratio=0.5)

# For the sequential/KB recommender
train_Xseq, test_Xseq, train_yseq, test_yseq = dataset.get_train_test_sequence_predictions()

Setting the train-test split by user...
Computing the training and test rating matrix...


131100it [00:17, 7504.06it/s]


Computing the test rating matrix split...
Computing the training and test rating matrix...


131100it [00:15, 8309.78it/s]
43119it [00:04, 9195.18it/s] 


Computing the training and test list of sequences...


131100it [00:07, 17888.19it/s]
43119it [00:02, 18762.90it/s]


In [18]:
print(f"Training (Content+CF): From {len(df_train_X)} ratings, predict the next {len(df_train_y)} for {df_train_y['reviewers'].nunique()} users")
print(f"Testing (Content+CF): From {len(df_test_X)} ratings, predict the next {len(df_test_y)} for {df_test_y['reviewers'].nunique()} users")

Training (Content+CF): From 119949 ratings, predict the next 11151 for 6631 users
Testing (Content+CF): From 157922 ratings, predict the next 16297 for 7680 users


In [20]:
print(f"Training (Sequential): From {len(train_Xseq)} sequences, predict the next course taken for {df_train_y['reviewers'].nunique()} users")
print(f"Testing (Sequential): From {len(test_Xseq)} sequences, predict the next course taken for {df_test_y['reviewers'].nunique()} users")

Training (Sequential): From 108061 sequences, predict the next course taken for 6631 users
Testing (Sequential): From 35439 sequences, predict the next course taken for 7680 users


## 1. Content-Based Recommendation

## 2. Collaborative Filtering Recommendation

## 3. Sequential/Knowledge-Based Recommendation

In [10]:
from coursemate.model import AssociationMiningModel
from coursemate.metrics import calculate_hit_rate_from_next_course_sequences

# cutoff determined empirically, testing the 90th and 99th percentile
model = AssociationMiningModel(cutoff=3, course_set=dataset.course_set)
df_seq = model.get_sequences_data(train_Xseq, train_yseq)
model.fit(df_seq)

100%|██████████████████████████████████████████████████████████████████████████| 23039/23039 [00:01<00:00, 17665.20it/s]


1-subsequences: 468
2-subsequences: 90472
3-subsequences: 1835231
(1926171, 2)


In [11]:
calculate_hit_rate_from_next_course_sequences(test_Xseq, test_yseq, model.recommend, 5)

100%|█████████████████████████████████████████████████████████████████████████████| 35439/35439 [06:47<00:00, 86.97it/s]


0.25906487203363526